In [1]:
%reset -sf
import numpy as np
import os
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing import Pool

In [2]:
os.chdir("/home/pj/projects/clifford_alg/data")
cliffords2 = np.load('cliffords_2Qnp.npy')
rtol=1e-03
atol=1e-05
L = 100

In [3]:
def flatten_and_uniquenize(clifford):
    flattened = clifford.ravel('C')
    for j in range(len(flattened)):
        if not np.isclose(0, flattened[j]):
            normalizer = np.conj(flattened[j]) / np.abs(flattened[j])
            break
        else:
            continue
    flattened = flattened * normalizer
    return flattened

In [4]:
flatten_and_uniquenize(cliffords2[5])

array([ 7.07106781e-01+0.00000000e+00j, -1.66533454e-16+7.07106781e-01j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        7.07106781e-01+1.66533454e-16j,  3.33066907e-16-7.07106781e-01j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        7.07106781e-01+0.00000000e+00j, -1.66533454e-16+7.07106781e-01j,
        0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        7.07106781e-01+1.66533454e-16j,  3.33066907e-16-7.07106781e-01j])

In [5]:
uniqueCliffords = []
for i in tqdm(range(len(cliffords2))):
    uniqueCliffords.append(flatten_and_uniquenize(cliffords2[i]))
uniqueCliffords = np.asarray(uniqueCliffords).ravel('C')

100%|██████████| 11520/11520 [00:00<00:00, 17412.63it/s]


In [6]:
def uniquenize_elements(flatarray):
    for i in tqdm(range(len(flatarray))):
        for j in range(i, len(flatarray)):
            if np.isclose(flatarray[i], flatarray[j], rtol, atol):
                flatarray[j] = flatarray[i]
    return np.unique(flatarray)

In [7]:
uniqueElements = np.unique(uniqueCliffords)
uniqueElements = uniquenize_elements(uniqueElements)

print("elements of 2d-clifford_group:{}\nunique elements:{}".format(len(cliffords2), len(uniqueElements)))

100%|██████████| 1369/1369 [00:24<00:00, 55.84it/s]

elements of 2d-clifford_group:11520
unique elements:13


In [8]:
cliffordsHash = []
for clifford in tqdm(cliffords2):
    flattened = flatten_and_uniquenize(clifford)
    cliffordHash = np.zeros(len(flattened), dtype = 'uint8')
    for i in range(len(flattened)):
        for j in range(len(uniqueElements)):
            if np.isclose(uniqueElements[j], flattened[i], rtol, atol):
                cliffordHash[i] = j + 1
                break;
    cliffordsHash.append(cliffordHash)

100%|██████████| 11520/11520 [00:42<00:00, 272.86it/s]


In [9]:
def hashnize(flattened, elementsHash):
    ret = np.zeros(len(flattened), dtype='int8')
    for i in range(len(flattened)):
        for j in range(len(elementsHash)):
            if np.isclose(elementsHash[j], flattened[i], rtol, atol):
                ret[i] = j + 1
                break
    return ret

In [10]:
def clifford_to_hash(clifford, elementHash):
    return hashnize(flatten_and_uniquenize(clifford), uniqueElements)

In [24]:
clifford_to_hash(cliffords2[1], uniqueElements)
hex1 = np.vectorize(lambda x: format(x, 'x'))(cliffordsHash[1].ravel('C'))
for i in range(len())

array(['a', 'd', 'a', 'a', 'd', 'a', 'a', 'a', 'a', 'a', 'a', 'd', 'a',
       'a', 'd', 'a'], dtype='<U1')

In [11]:
def process(i):
    ret = []
    for j in range(L):
        producted = np.matmul(cliffords2[i], cliffords2[j])
        productedHash = hashnize(flatten_and_uniquenize(producted), uniqueElements)
        for k in range(len(cliffordsHash)):
            if np.array_equal(cliffordsHash[k], productedHash):
                ret.append(k + 1)
                break
            else:
                continue
    return ret

In [12]:
p = Pool()
resultMap = np.asarray(p.map(process, range(L)))

In [14]:
print(resultMap)

[[  1   2   3 ...  98  99 100]
 [  2   1   4 ...  97 100  99]
 [  3   4   1 ... 100  97  98]
 ...
 [ 98  97 100 ... 265 268 267]
 [ 99 100  97 ... 268 265 266]
 [100  99  98 ... 267 266 265]]


In [15]:
np.savetxt("cliffords.csv", resultMap, delimiter=',' ,fmt = '%i')

In [41]:
map = np.zeros((L, L), dtype='int8')
for i in tqdm(range(L)):
    for j in range(L):
        producted = np.matmul(cliffords2[i], cliffords2[j])
        productedHash = hashnize(flatten_and_uniquenize(producted), uniqueElements)
        for k in range(len(cliffordsHash)):
            if np.array_equal(cliffordsHash[k], productedHash):
                map[i][j] = k + 1
                break
            else:
                continue

 12%|█▏        | 12/100 [00:05<00:39,  2.23it/s]


KeyboardInterrupt: 